In [179]:
# ANN Training
import numpy as np
import pandas as pd
import random
import copy
import time
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization
from tensorflow.keras.layers import Dropout


In [57]:
# NO NEED TO LOAD / READIN

# READ IN CSV (takes 30 seconds)
new_train_values = np.loadtxt("train_vectors2.csv",delimiter=",")
print(new_train_values.shape)
new_train = new_train_values.reshape((4000,16385))
print('new_train.shape = ',new_train.shape)


new_test_values = np.loadtxt("test_vectors2.csv",delimiter=",")
print(new_test_values.shape)
new_test = new_test_values.reshape((4000,16385))
print('new_test.shape = ',new_test.shape)

train_x = new_train[:,0:16384] 
train_y = new_train[:,16384]
print(train_x.shape)
print(train_y.shape)
#===================
test_x = new_test[:,0:16384] 
test_y = new_test[:,16384]
print(test_x.shape)
print(test_y.shape)

(4000, 16385)
new_train.shape =  (4000, 16385)
(4000, 16385)
new_test.shape =  (4000, 16385)


In [59]:
from keras.losses import binary_crossentropy

# Compute Binary Cross-Entropy using NumPy
def binary_cross_entropy_np(y_true, y_pred):
    """
    Calculates Binary Cross-Entropy loss for multiple samples using NumPy.
    y_true: NumPy array of actual labels (0s and 1s)
    y_pred: NumPy array of predicted probabilities (between 0 and 1)
    """
    epsilon = 1e-15  # Small value to prevent log(0)
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)  # Clip probabilities
    loss = -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
    return loss



# Compute Binary Cross-Entropy using own calculaton for checking
def binary_cross_entropy_check(y_true, y_pred):
    """
    Calculates Binary Cross-Entropy loss for multiple samples using NumPy.
    y_true: NumPy array of actual labels (0s and 1s)
    y_pred: NumPy array of predicted probabilities (between 0 and 1)
    """
    epsilon = 1e-15  # Small value to prevent log(0)
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)  # Clip probabilities
    
    N=len(y_true)
    BCE_sum=0
    for i in range(N):
        BCE_sum = BCE_sum + y_true[i]*np.log(y_pred[i]) + (1-y_true[i])*np.log((1-y_pred[i])) 

    BCE = (-1*BCE_sum)/N
    
    return BCE


In [41]:
# START RUNNING THE CODE BELOW 

In [195]:
# To define some ANN models

def baseline_model():
    model = Sequential()
    model.add(Dense(512, activation='relu', input_dim = 20))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(32, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(1, activation='sigmoid')) # Output layer for binary classification
    
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    
    return model

def baseline_model2():
    model = Sequential()
    model.add(Dense(32, activation='relu', input_dim = 20))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(8, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(1, activation='sigmoid')) # Output layer for binary classification
    
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    
    return model


# Build the model
# ANN_model = baseline_model()

ANN_model = baseline_model()

ANN_model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_33 (Dense)             (None, 512)               10752     
_________________________________________________________________
batch_normalization_24 (Batc (None, 512)               2048      
_________________________________________________________________
dropout_15 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_34 (Dense)             (None, 128)               65664     
_________________________________________________________________
batch_normalization_25 (Batc (None, 128)               512       
_________________________________________________________________
dropout_16 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_35 (Dense)             (None, 32)               

In [176]:
# To create the balanced training and testing dataset
# 40 cases

In [196]:
print('Ref vector  : ---   ---   ---   ---   ---   ---   ---   ---   ---   ---')
print('Var vector  : -C-   ---   ---   ---   ---   ---   ---   ---   ---   ---  label 1')
print('Var vector  : ---   ---   -C-   ---   ---   ---   ---   ---   ---   ---  label 1')
print('Var vector  : ---   ---   ---   ---   -C-   ---   ---   ---   ---   ---  label 1')
print('Var vector  : ---   ---   ---   ---   ---   ---   -C-   ---   ---   ---  label 1')
print('Var vector  : ---   ---   ---   ---   ---   ---   ---   ---   -C-   ---  label 1')
print('Var vector  : -C-   ---   -C-   ---   ---   ---   ---   ---   ---   ---  label 1')
print('Var vector  : ---   ---   -C-   ---   ---   ---   -C-   ---   ---   ---  label 1')
print('Var vector  : ---   ---   ---   ---   -C-   ---   ---   ---   -C-   ---  label 1')
print('Var vector  : -C-   ---   ---   ---   ---   ---   -C-   ---   ---   ---  label 1')
print('Var vector  : ---   ---   -C-   ---   ---   ---   ---   ---   -C-   ---  label 1')

print('Var vector  : -C-   ---   ---   ---   -C-   ---   ---   ---   -C-   ---  label 1')
print('Var vector  : ---   ---   -C-   ---   -C-   ---   ---   ---   -C-   ---  label 1')
print('Var vector  : ---   ---   -C-   ---   -C-   ---   -C-   ---   ---   ---  label 1')
print('Var vector  : ---   ---   -C-   ---   ---   ---   -C-   ---   -C-   ---  label 1')
print('Var vector  : -C-   ---   ---   ---   -C-   ---   -C-   ---   ---   ---  label 1')
print('Var vector  : -C-   ---   -C-   ---   -C-   ---   -C-   ---   ---   ---  label 1')
print('Var vector  : -C-   ---   -C-   ---   -C-   ---   ---   ---   -C-   ---  label 1')
print('Var vector  : -C-   ---   -C-   ---   ---   ---   -C-   ---   -C-   ---  label 1')
print('Var vector  : -C-   ---   ---   ---   -C-   ---   -C-   ---   -C-   ---  label 1')
print('Var vector  : -C-   ---   -C-   ---   -C-   ---   -C-   ---   -C-   ---  label 1')
print('================================================================================')
print('Var vector  : ---   -C-   ---   ---   ---   ---   ---   ---   ---   ---  label 0')
print('Var vector  : ---   ---   ---   -C-   ---   ---   ---   ---   ---   ---  label 0')
print('Var vector  : ---   ---   ---   ---   ---   -C-   ---   ---   ---   ---  label 0')
print('Var vector  : ---   ---   ---   ---   ---   ---   ---   -C-   ---   ---  label 0')
print('Var vector  : ---   ---   ---   ---   ---   ---   ---   ---   ---   -C-  label 0')
print('Var vector  : ---   -C-   ---   -C-   ---   ---   ---   ---   ---   ---  label 0')
print('Var vector  : ---   ---   ---   -C-   ---   ---   ---   -C-   ---   ---  label 0')
print('Var vector  : ---   ---   ---   ---   ---   -C-   ---   ---   ---   -C-  label 0')
print('Var vector  : ---   -C-   ---   ---   ---   ---   ---   -C-   ---   ---  label 0')
print('Var vector  : ---   ---   ---   -C-   ---   ---   ---   ---   ---   -C-  label 0')

print('Var vector  : ---   -C-   ---   ---   ---   -C-   ---   ---   ---   -C-  label 0')
print('Var vector  : ---   ---   ---   -C-   ---   -C-   ---   ---   ---   -C-  label 0')
print('Var vector  : ---   ---   ---   -C-   ---   -C-   ---   -C-   ---   ---  label 0')
print('Var vector  : ---   ---   ---   -C-   ---   ---   ---   -C-   ---   -C-  label 0')
print('Var vector  : ---   -C-   ---   ---   ---   -C-   ---   -C-   ---   ---  label 0')
print('Var vector  : ---   -C-   ---   -C-   ---   -C-   ---   -C-   ---   ---  label 0')
print('Var vector  : ---   -C-   ---   -C-   ---   -C-   ---   ---   ---   -C-  label 0')
print('Var vector  : ---   -C-   ---   -C-   ---   ---   ---   -C-   ---   -C-  label 0')
print('Var vector  : ---   -C-   ---   ---   ---   -C-   ---   -C-   ---   -C-  label 0')
print('Var vector  : ---   ---   ---   -C-   ---   -C-   ---   -C-   ---   -C-  label 0')


Ref vector  : ---   ---   ---   ---   ---   ---   ---   ---   ---   ---
Var vector  : -C-   ---   ---   ---   ---   ---   ---   ---   ---   ---  label 1
Var vector  : ---   ---   -C-   ---   ---   ---   ---   ---   ---   ---  label 1
Var vector  : ---   ---   ---   ---   -C-   ---   ---   ---   ---   ---  label 1
Var vector  : ---   ---   ---   ---   ---   ---   -C-   ---   ---   ---  label 1
Var vector  : ---   ---   ---   ---   ---   ---   ---   ---   -C-   ---  label 1
Var vector  : -C-   ---   -C-   ---   ---   ---   ---   ---   ---   ---  label 1
Var vector  : ---   ---   -C-   ---   ---   ---   -C-   ---   ---   ---  label 1
Var vector  : ---   ---   ---   ---   -C-   ---   ---   ---   -C-   ---  label 1
Var vector  : -C-   ---   ---   ---   ---   ---   -C-   ---   ---   ---  label 1
Var vector  : ---   ---   -C-   ---   ---   ---   ---   ---   -C-   ---  label 1
Var vector  : -C-   ---   ---   ---   -C-   ---   ---   ---   -C-   ---  label 1
Var vector  : ---   ---   -C-   ---  

In [197]:
# To create the dataset
def generate_dataset(num_items):
    train_x = np.empty((0,20))
    train_y = np.empty((0,1))
    lowerB = 0.99
    upperB = 1.01
    lowerR = 1.2  # modify individual element of variant vector
    upperR = 1.5  # in range [1.2, 1.5]

    for i in range(0,num_items):
        #print('*** i = ',i)
        #Ref vector
        ref_vector1 = np.array(np.random.uniform(low=0.01, high=0.6, size=10))
        # Initial variant is obtained from deepcopy
        k_adjust = random.uniform(lowerB,upperB)

        var_vector1 = copy.deepcopy(ref_vector1)*k_adjust

        index = (i % 40)


        if (index == 0): 
            pos=[0]
            zzz_y = np.array([1])
        if (index == 1): 
            pos=[2]  
            zzz_y = np.array([1])
        if (index == 2): 
            pos=[4]  
            zzz_y = np.array([1])
        if (index == 3): 
            pos=[6]  
            zzz_y = np.array([1])
        if (index == 4): 
            pos=[8]  
            zzz_y = np.array([1])
        if (index == 5): 
            pos=[0, 2]
            zzz_y = np.array([1])
        if (index == 6): 
            pos=[2, 6]  
            zzz_y = np.array([1])
        if (index == 7): 
            pos=[4, 8]  
            zzz_y = np.array([1])
        if (index == 8): 
            pos=[0, 6]  
            zzz_y = np.array([1])
        if (index == 9): 
            pos=[2, 8]  
            zzz_y = np.array([1])

        if (index == 10): 
            pos=[0, 4, 8]
            zzz_y = np.array([1])
        if (index == 11): 
            pos=[2, 4, 8]  
            zzz_y = np.array([1])
        if (index == 12): 
            pos=[2, 4, 6]  
            zzz_y = np.array([1])
        if (index == 13): 
            pos=[2, 6, 8]  
            zzz_y = np.array([1])
        if (index == 14): 
            pos=[0, 4, 6]  
            zzz_y = np.array([1])
        if (index == 15): 
            pos=[0, 2, 4, 6]
            zzz_y = np.array([1])
        if (index == 16): 
            pos=[0, 2, 4, 8]  
            zzz_y = np.array([1])
        if (index == 17): 
            pos=[0, 2, 6, 8]  
            zzz_y = np.array([1])
        if (index == 18): 
            pos=[0, 4, 6, 8]  
            zzz_y = np.array([1])
        if (index == 19): 
            pos=[0, 2, 4, 6, 8]  
            zzz_y = np.array([1])


        if (index == 20): 
            pos=[1]
            zzz_y = np.array([0])
        if (index == 21): 
            pos=[3]  
            zzz_y = np.array([0])
        if (index == 22): 
            pos=[5]  
            zzz_y = np.array([0])
        if (index == 23): 
            pos=[7]  
            zzz_y = np.array([0])
        if (index == 24): 
            pos=[9]  
            zzz_y = np.array([0])
        if (index == 25): 
            pos=[1, 3]
            zzz_y = np.array([0])
        if (index == 26): 
            pos=[3, 7]  
            zzz_y = np.array([0])
        if (index == 27): 
            pos=[5, 9]  
            zzz_y = np.array([0])
        if (index == 28): 
            pos=[1, 7]  
            zzz_y = np.array([0])
        if (index == 29): 
            pos=[3, 9]  
            zzz_y = np.array([0])


        if (index == 30): 
            pos=[1, 5, 9]
            zzz_y = np.array([0])
        if (index == 31): 
            pos=[3, 5, 9]  
            zzz_y = np.array([0])
        if (index == 32): 
            pos=[3, 5, 7]  
            zzz_y = np.array([0])
        if (index == 33): 
            pos=[3, 7, 9]  
            zzz_y = np.array([0])
        if (index == 34): 
            pos=[1, 5, 7]  
            zzz_y = np.array([0])
        if (index == 35): 
            pos=[1, 3, 5, 7]
            zzz_y = np.array([0])
        if (index == 36): 
            pos=[1, 3, 5, 9]  
            zzz_y = np.array([0])
        if (index == 37): 
            pos=[1, 3, 7, 9]  
            zzz_y = np.array([0])
        if (index == 38): 
            pos=[1, 5, 7, 9]  
            zzz_y = np.array([0])
        if (index == 39): 
            pos=[1, 3, 5, 7, 9]  
            zzz_y = np.array([0])



        kk = random.uniform(lowerR,upperR)
        for j in pos:
            var_vector1[j] = ref_vector1[j]*kk

        zzz_x = np.concatenate((ref_vector1,var_vector1))   

        train_x = np.append(train_x,[zzz_x],axis=0)
        train_y = np.append(train_y,[zzz_y],axis=0)

        # np.set_printoptions(precision=4)
        # print(zzz)
    
    return train_x, train_y
 


In [198]:
train_x, train_y = generate_dataset(2000)
print(train_x.shape)
print(train_y.shape)
test_x, test_y = generate_dataset(2000)

(2000, 20)
(2000, 1)


In [199]:
# training

start_time = time.time()

history = ANN_model.fit(train_x, train_y, epochs=50, batch_size=64,validation_split=0.15,verbose = 2)

end_time = time.time()
exe_time = end_time - start_time
print("Execution time: ", exe_time)

scores = ANN_model.evaluate(test_x,test_y,verbose = 2)

Epoch 1/50
27/27 - 1s - loss: 0.6611 - accuracy: 0.6447 - val_loss: 0.6861 - val_accuracy: 0.4733
Epoch 2/50
27/27 - 0s - loss: 0.3966 - accuracy: 0.8100 - val_loss: 0.6507 - val_accuracy: 0.7467
Epoch 3/50
27/27 - 0s - loss: 0.2694 - accuracy: 0.8818 - val_loss: 0.5974 - val_accuracy: 0.7333
Epoch 4/50
27/27 - 0s - loss: 0.2031 - accuracy: 0.9194 - val_loss: 0.5483 - val_accuracy: 0.8533
Epoch 5/50
27/27 - 0s - loss: 0.1764 - accuracy: 0.9312 - val_loss: 0.5106 - val_accuracy: 0.7200
Epoch 6/50
27/27 - 0s - loss: 0.1464 - accuracy: 0.9388 - val_loss: 0.4367 - val_accuracy: 0.8767
Epoch 7/50
27/27 - 0s - loss: 0.1337 - accuracy: 0.9506 - val_loss: 0.3890 - val_accuracy: 0.8700
Epoch 8/50
27/27 - 0s - loss: 0.1161 - accuracy: 0.9612 - val_loss: 0.3807 - val_accuracy: 0.8200
Epoch 9/50
27/27 - 0s - loss: 0.1069 - accuracy: 0.9612 - val_loss: 0.3899 - val_accuracy: 0.7433
Epoch 10/50
27/27 - 0s - loss: 0.0896 - accuracy: 0.9671 - val_loss: 0.2443 - val_accuracy: 0.9467
Epoch 11/50
27/27 -

In [200]:
# To get the results of the ANN using test dataset
y_pred = ANN_model.predict(test_x)

print('  max and min ')
print(np.max(y_pred))
print(np.min(y_pred))

epsilon = 1e-15  # Small value to prevent log(0)
y_pred = np.clip(y_pred, epsilon, 1 - epsilon)  # Clip probabilities

print(test_x.shape)
print(test_y.shape)
print(y_pred.shape)

total_loss = binary_cross_entropy_check(test_y, y_pred)
print(f"** CHECK Average BCE Loss for multiple samples: {total_loss}")

# To check on accuracy
# first, convert the elements in y_pred so that negative becones epsilon, largest is 1 - epsilon
epsilon = 1e-15  # Small value to prevent log(0)
y_pred = np.clip(y_pred, epsilon, 1 - epsilon)  # Clip probabilities
N=len(test_y)
threshold = 0.5
y_pred_binary = (y_pred >= threshold).astype(int)

accuracy = accuracy_score(test_y,y_pred_binary)
print("Accuracy Score:",accuracy)

#Count error cases:
#
test_y_binary = test_y
error0to1 = 0  # test_y is 0
error1to0 = 0  # test_y is 1
for i in range(N):
    if (test_y_binary[i] == 0 and y_pred_binary[i] == 1):
        error0to1 += 1
    if (test_y_binary[i] == 1 and y_pred_binary[i] == 0):
        error1to0 += 1
        
print('error0to1 = ',error0to1, '; label is 0')
print('error1to0 = ',error1to0, '; label is 1')
print('total error = ',error0to1+error1to0,'percentError = ', 100*(error0to1+error1to0)/N)
print('2000 test cases: 1000 label 0; 1000 label 1')

  max and min 
0.9999995
1.3749329e-08
(2000, 20)
(2000, 1)
(2000, 1)
** CHECK Average BCE Loss for multiple samples: [0.0438]
Accuracy Score: 0.983
error0to1 =  7 ; label is 0
error1to0 =  27 ; label is 1
total error =  34 percentError =  1.7
4000 test cases: 2000 label 0; 2000 label 1


In [204]:
print('test_x.shape ', test_x.shape)
print('test_y.shape ',test_y.shape)
print(y_pred_binary.shape)

print('Compare between test_y and y_pred_binary')

errorcase=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

for i in range(0,2000):
    index = (i % 40)
    if (index == 0 and test_y[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
    if (index == 1 and test_y[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
    if (index == 2 and test_y[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
    if (index == 3 and test_y[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
    if (index == 4 and test_y[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
    if (index == 5 and test_y[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
    if (index == 6 and test_y[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
    if (index == 7 and test_y[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
    if (index == 8 and test_y[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
    if (index == 9 and test_y[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
    if (index == 10 and test_y[i] != y_pred_binary[i] ): 
        errorcase[index] = errorcase[index]+1
    if (index == 11 and test_y[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
    if (index == 12 and test_y[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
    if (index == 13 and test_y[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
    if (index == 14 and test_y[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
    if (index == 15 and test_y[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
    if (index == 16 and test_y[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
    if (index == 17 and test_y[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
    if (index == 18 and test_y[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
    if (index == 19 and test_y[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
    if (index == 20 and test_y[i] != y_pred_binary[i] ): 
        errorcase[index] = errorcase[index]+1
    if (index == 21 and test_y[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
    if (index == 22 and test_y[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
    if (index == 23 and test_y[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
    if (index == 24 and test_y[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
    if (index == 25 and test_y[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
    if (index == 26 and test_y[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
    if (index == 27 and test_y[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
    if (index == 28 and test_y[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
    if (index == 29 and test_y[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
    if (index == 30 and test_y[i] != y_pred_binary[i] ): 
        errorcase[index] = errorcase[index]+1
    if (index == 31 and test_y[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
    if (index == 32 and test_y[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
    if (index == 33 and test_y[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
    if (index == 34 and test_y[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
    if (index == 35 and test_y[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
    if (index == 36 and test_y[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
    if (index == 37 and test_y[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
    if (index == 38 and test_y[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
    if (index == 39 and test_y[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1       

print(errorcase)

test_x.shape  (2000, 20)
test_y.shape  (2000, 1)
(2000, 1)
Compare between test_y and y_pred_binary
[2, 1, 8, 5, 10, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 3, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]


In [206]:
print(len(errorcase))
print('Sum of errors = ' , np.sum(errorcase))

40
Sum of errors =  34


In [207]:
# To get the results of the ANN using train dataset
y_pred = ANN_model.predict(train_x)

print(train_x.shape)
print(train_y.shape)
print(y_pred.shape)

total_loss = binary_cross_entropy_check(train_y, y_pred)
print(f"** CHECK Average BCE Loss for multiple samples: {total_loss}")

# To check on accuracy
# first, convert the elements in y_pred so that negative becones epsilon, largest is 1 - epsilon
epsilon = 1e-15  # Small value to prevent log(0)
y_pred = np.clip(y_pred, epsilon, 1 - epsilon)  # Clip probabilities

N = len(train_y)
threshold = 0.5
y_pred_binary = (y_pred >= threshold).astype(int)

accuracy = accuracy_score(train_y,y_pred_binary)
print("Accuracy Score:",accuracy)

#Count error cases:
#
train_y_binary = train_y
error0to1 = 0  # test_y is 0
error1to0 = 0  # test_y is 1
for i in range(N):
    if (train_y_binary[i] == 0 and y_pred_binary[i] == 1):
        error0to1 += 1
    if (train_y_binary[i] == 1 and y_pred_binary[i] == 0):
        error1to0 += 1
        
print('error0to1 = ',error0to1, '; label is 0')
print('error1to0 = ',error1to0, '; label is 1')
print('total error = ',error0to1+error1to0,'percentError = ', 100*(error0to1+error1to0)/N)
print('train cases: ', len(train_y))

(2000, 20)
(2000, 1)
(2000, 1)
** CHECK Average BCE Loss for multiple samples: [0.0201]
Accuracy Score: 0.9935
error0to1 =  3 ; label is 0
error1to0 =  10 ; label is 1
total error =  13 percentError =  0.65
train cases:  2000


In [111]:
# DELETE below
# Part 2 of investigation
# 
#
DBset = np.empty((0,8))

for i in range(0,2000):
    #Ref vector
    aa=np.mean(train_x[i,0:2000])
    bb=np.mean(train_x[i,2000:4000])
    cc=np.mean(train_x[i,4000:6000])
    dd=np.mean(train_x[i,6000:8000])
    #Variant vector
    ee = np.mean(train_x[i,8192:10192])
    ff = np.mean(train_x[i,10192:12192])
    gg = np.mean(train_x[i,12192:14192])
    hh = np.mean(train_x[i,14192:16192])
    
    index = (i % 8)
    # index 0,1,2,3 are labe 1; index 4,5,6,7 are labe 0; 
    lowerR = 1.1
    upperR = 1.45
    
    if (index == 0): 
        kk = random.uniform(lowerR,upperR)
        zzz = np.array([aa, bb, cc, dd, ee, ff, gg, hh*kk])
    if (index == 1):
        kk = random.uniform(lowerR,upperR)
        zzz = np.array([aa, bb, cc, dd, ee, ff*kk, gg, hh])
    if (index == 2):
        kk = random.uniform(lowerR,upperR)
        zzz = np.array([aa, bb, cc, dd, ee, ff*kk, gg, hh*kk])
    if (index == 3):
        kk = random.uniform(lowerR,upperR)
        zzz = np.array([aa, bb, cc, dd, ee*kk, ff*kk, gg*kk, hh*kk])   
        
    if (index == 4):
        kk = random.uniform(lowerR,upperR)
        zzz = np.array([aa, bb, cc, dd, ee, ff, gg*kk, hh])
    if (index == 5):
        kk = random.uniform(lowerR,upperR)
        zzz = np.array([aa, bb, cc, dd, ee*kk, ff, gg, hh])
    if (index == 6):
        kk = random.uniform(lowerR,upperR)
        zzz = np.array([aa, bb, cc, dd, ee*kk, ff, gg*kk, hh])
    if (index == 7):
        kk = random.uniform(lowerR,upperR)
        zzz = np.array([aa, bb, cc, dd, ee, ff, gg, hh])  
    
    DBset = np.append(DBset,[zzz],axis=0)
    # np.set_printoptions(precision=4)
    # print(zzz)
    

 


[[0.3039 0.3006 0.3049 ... 0.3006 0.3049 0.4289]
 [0.3045 0.3033 0.3086 ... 0.3599 0.3086 0.3018]
 [0.3049 0.3029 0.3019 ... 0.3978 0.3019 0.4091]
 ...
 [0.3108 0.3029 0.2993 ... 0.3029 0.2993 0.3026]
 [0.3005 0.3072 0.3018 ... 0.3072 0.4452 0.3074]
 [0.3052 0.3038 0.3063 ... 0.3038 0.3062 0.2989]]


In [128]:
# Create the test set of 2000

DBset_test = np.empty((0,8))

for i in range(0,2000):
    #Ref vector
    aa=np.mean(test_x[i,0:2000])
    bb=np.mean(test_x[i,2000:4000])
    cc=np.mean(test_x[i,4000:6000])
    dd=np.mean(test_x[i,6000:8000])
    #Variant vector
    ee = np.mean(test_x[i,8192:10192])
    ff = np.mean(test_x[i,10192:12192])
    gg = np.mean(test_x[i,12192:14192])
    hh = np.mean(test_x[i,14192:16192])
    
    index = (i % 8)
    # index 0,1,2,3 are labe 1; index 4,5,6,7 are labe 0; 
    lowerR = 1.1
    upperR = 1.45
    
    if (index == 0): 
        kk = random.uniform(lowerR,upperR)
        zzz = np.array([aa, bb, cc, dd, ee, ff, gg, hh*kk])
    if (index == 1):
        kk = random.uniform(lowerR,upperR)
        zzz = np.array([aa, bb, cc, dd, ee, ff*kk, gg, hh])
    if (index == 2):
        kk = random.uniform(lowerR,upperR)
        zzz = np.array([aa, bb, cc, dd, ee, ff*kk, gg, hh*kk])
    if (index == 3):
        kk = random.uniform(lowerR,upperR)
        zzz = np.array([aa, bb, cc, dd, ee*kk, ff*kk, gg*kk, hh*kk])   
        
    if (index == 4):
        kk = random.uniform(lowerR,upperR)
        zzz = np.array([aa, bb, cc, dd, ee, ff, gg*kk, hh])
    if (index == 5):
        kk = random.uniform(lowerR,upperR)
        zzz = np.array([aa, bb, cc, dd, ee*kk, ff, gg, hh])
    if (index == 6):
        kk = random.uniform(lowerR,upperR)
        zzz = np.array([aa, bb, cc, dd, ee*kk, ff, gg*kk, hh])
    if (index == 7):
        kk = random.uniform(lowerR,upperR)
        zzz = np.array([aa, bb, cc, dd, ee, ff, gg, hh])  
    
    DBset_test = np.append(DBset_test,[zzz],axis=0)
    # np.set_printoptions(precision=4)
    # print(zzz)
    

 


In [ ]:
# Do not run
# debug
#
DBset_test = np.empty((0,8))

for i in range(0,2000):
    #Ref vector
    aa=np.mean(test_x[i,0:2000])
    bb=np.mean(test_x[i,2000:4000])
    cc=np.mean(test_x[i,4000:6000])
    dd=np.mean(test_x[i,6000:8000])
    #Variant vector
    ee = np.mean(test_x[i,8192:10192])
    ff = np.mean(test_x[i,10192:12192])
    gg = np.mean(test_x[i,12192:14192])
    hh = np.mean(test_x[i,14192:16192])
    
    index = (i % 8)
    # index 0,1,2,3 are labe 1; index 4,5,6,7 are labe 0; 
    lowerR = 1.1
    upperR = 1.45
    
    if (index == 0): 
        kk = random.uniform(lowerR,upperR)
        zzz = np.array([aa, bb, cc, dd, ee, ff, gg, hh*kk])
    if (index == 1):
        kk = random.uniform(lowerR,upperR)
        zzz = np.array([aa, bb, cc, dd, ee, ff*kk, gg, hh])
    if (index == 2):
        kk = random.uniform(lowerR,upperR)
        zzz = np.array([aa, bb, cc, dd, ee, ff*kk, gg, hh*kk])
    if (index == 3):
        kk = random.uniform(lowerR,upperR)
        zzz = np.array([aa, bb, cc, dd, ee*kk, ff*kk, gg*kk, hh*kk])   
        
    if (index == 4):
        kk = random.uniform(lowerR,upperR)
        zzz = np.array([aa, bb, cc, dd, ee, ff, gg*kk, hh])
    if (index == 5):
        kk = random.uniform(lowerR,upperR)
        zzz = np.array([aa, bb, cc, dd, ee*kk, ff, gg, hh])
    if (index == 6):
        kk = random.uniform(lowerR,upperR)
        zzz = np.array([aa, bb, cc, dd, ee*kk, ff, gg*kk, hh])
    if (index == 7):
        kk = random.uniform(lowerR,upperR)
        zzz = np.array([aa, bb, cc, dd, ee, ff, gg, hh])  
    
    DBset_test = np.append(DBset_test,[zzz],axis=0)
    # np.set_printoption

In [129]:
    
    print(DBset.shape)
    DBy = train_y[0:2000]
    print(DBy.shape)
    
    print(DBy[0:16])
    
    print(DBset_test.shape)
        
    
    

(2000, 8)
(2000,)
[1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0.]
(2000, 8)


In [130]:
import os
DIR = "C:/Users/gpang/iCloudDrive/Naja/20250628-Evo2/code"
out_path = os.path.join(DIR, "trainX_vectors3.csv")
print(out_path)
# DB_train.tofile(out_path,sep=',',fmt='%.5f')
np.savetxt(out_path,DBset,delimiter=',',fmt='%.5f')
print("Finished Export of Training cases ")

C:/Users/gpang/iCloudDrive/Naja/20250628-Evo2/code\trainX_vectors3.csv
Finished Export of Training cases 


In [131]:
out_path = os.path.join(DIR, "trainY_vectors3.csv")
print(out_path)
# DB_test.tofile(out_path,sep=',',fmt='#.5f')
np.savetxt(out_path,DBy,delimiter=',',fmt='%.5f')
print("Finished Export of Testing cases ")

C:/Users/gpang/iCloudDrive/Naja/20250628-Evo2/code\trainY_vectors3.csv
Finished Export of Testing cases 


In [132]:
out_path = os.path.join(DIR, "testX_vectors3.csv")
print(out_path)
# DB_train.tofile(out_path,sep=',',fmt='%.5f')
np.savetxt(out_path,DBset_test,delimiter=',',fmt='%.5f')
print("Finished Export of Training cases ")

C:/Users/gpang/iCloudDrive/Naja/20250628-Evo2/code\testX_vectors3.csv
Finished Export of Training cases 


In [133]:
def baseline_model3():
    model = Sequential()
    model.add(Dense(32, activation='relu', input_dim = 8))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(8, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(1, activation='sigmoid')) # Output layer for binary classification
    
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    
    return model


ANN_model = baseline_model3()

ANN_model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_30 (Dense)             (None, 32)                288       
_________________________________________________________________
batch_normalization_22 (Batc (None, 32)                128       
_________________________________________________________________
dropout_14 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_31 (Dense)             (None, 8)                 264       
_________________________________________________________________
batch_normalization_23 (Batc (None, 8)                 32        
_________________________________________________________________
dense_32 (Dense)             (None, 1)                 9         
Total params: 721
Trainable params: 641
Non-trainable params: 80
_______________________________________________________

In [150]:
start_time = time.time()

history = ANN_model.fit(DBset, DBy, epochs=40, batch_size=64,validation_split=0.2,verbose = 2)

end_time = time.time()
exe_time = end_time - start_time
print("Execution time: ", exe_time)


Epoch 1/40
25/25 - 1s - loss: 0.8202 - accuracy: 0.4313 - val_loss: 0.6919 - val_accuracy: 0.5000
Epoch 2/40
25/25 - 0s - loss: 0.5825 - accuracy: 0.7013 - val_loss: 0.6846 - val_accuracy: 0.5000
Epoch 3/40
25/25 - 0s - loss: 0.4539 - accuracy: 0.8669 - val_loss: 0.6780 - val_accuracy: 0.5000
Epoch 4/40
25/25 - 0s - loss: 0.3591 - accuracy: 0.9306 - val_loss: 0.6705 - val_accuracy: 0.5000
Epoch 5/40
25/25 - 0s - loss: 0.2948 - accuracy: 0.9444 - val_loss: 0.6604 - val_accuracy: 0.5000
Epoch 6/40
25/25 - 0s - loss: 0.2442 - accuracy: 0.9494 - val_loss: 0.6488 - val_accuracy: 0.5000
Epoch 7/40
25/25 - 0s - loss: 0.1802 - accuracy: 0.9731 - val_loss: 0.6314 - val_accuracy: 0.5000
Epoch 8/40
25/25 - 0s - loss: 0.1544 - accuracy: 0.9825 - val_loss: 0.6261 - val_accuracy: 0.5000
Epoch 9/40
25/25 - 0s - loss: 0.1425 - accuracy: 0.9750 - val_loss: 0.6233 - val_accuracy: 0.5000
Epoch 10/40
25/25 - 0s - loss: 0.1214 - accuracy: 0.9825 - val_loss: 0.6233 - val_accuracy: 0.5000
Epoch 11/40
25/25 -

In [168]:
print('Ref vector  : ---   ---   ---   --- ')
print('Var index 0 : ---   ---   ---   -C- ', ' label 1')
print('Var index 1 : ---   -C-   ---   --- ', ' label 1')
print('Var index 2 : ---   -C-   ---   -C- ', ' label 1')
print('Var index 3 : -C-   -C-   -C-   -C- ', ' 70% label 1')

print('Var index 4 : ---   ---   -C-   --- ', ' label 0')
print('Var index 5 : -C-   ---   ---   --- ', ' label 0')
print('Var index 6 : -C-   ---   -C-   --- ', ' label 0')
print('Var index 7 : -C-   -C-   -C-   -C- ', ' 0.3% label 0')

print(DBset_test.shape)
print(DBy.shape)
print(DBy[0:16])
print(DBset_test[0:8,0:8])

Ref vector  : ---   ---   ---   --- 
Var index 0 : ---   ---   ---   -C-   label 1
Var index 1 : ---   -C-   ---   ---   label 1
Var index 2 : ---   -C-   ---   -C-   label 1
Var index 3 : -C-   -C-   -C-   -C-   70% label 1
Var index 4 : ---   ---   -C-   ---   label 0
Var index 5 : -C-   ---   ---   ---   label 0
Var index 6 : -C-   ---   -C-   ---   label 0
Var index 7 : -C-   -C-   -C-   -C-   0.3% label 0
(2000, 8)
(2000,)
[1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0.]
[[0.2998 0.3119 0.3061 0.3032 0.2998 0.3119 0.3061 0.4311]
 [0.3094 0.3061 0.3012 0.3067 0.3094 0.374  0.3012 0.3067]
 [0.3065 0.3079 0.3024 0.3035 0.3065 0.4077 0.3024 0.4018]
 [0.3021 0.3061 0.3095 0.3106 0.3375 0.3403 0.3473 0.3523]
 [0.3054 0.3114 0.3052 0.2988 0.3054 0.3114 0.3381 0.2988]
 [0.3064 0.3077 0.3141 0.3042 0.3998 0.3077 0.3141 0.3042]
 [0.3043 0.304  0.3102 0.3022 0.4186 0.304  0.42   0.3022]
 [0.3065 0.3089 0.3051 0.3037 0.3065 0.3091 0.3052 0.3035]]


In [155]:
# To get the results of the ANN using test dataset
y_pred = ANN_model.predict(DBset_test)

print('  max and min ')
print(np.max(y_pred))
print(np.min(y_pred))

epsilon = 1e-15  # Small value to prevent log(0)
y_pred = np.clip(y_pred, epsilon, 1 - epsilon)  # Clip probabilities

print(DBset_test.shape)
print(DBy.shape)
print(y_pred.shape)

total_loss = binary_cross_entropy_check(DBy, y_pred)
print(f"** CHECK Average BCE Loss for multiple samples: {total_loss}")

# To check on accuracy
# first, convert the elements in y_pred so that negative becones epsilon, largest is 1 - epsilon
epsilon = 1e-15  # Small value to prevent log(0)
y_pred = np.clip(y_pred, epsilon, 1 - epsilon)  # Clip probabilities
N=len(DBy)
threshold = 0.5
y_pred_binary = (y_pred >= threshold).astype(int)

accuracy = accuracy_score(DBy,y_pred_binary)
print("Accuracy Score:",accuracy)

#Count error cases:
#
test_y_binary = DBy
error0to1 = 0  # test_y is 0
error1to0 = 0  # test_y is 1
for i in range(N):
    if (test_y_binary[i] == 0 and y_pred_binary[i] == 1):
        error0to1 += 1
    if (test_y_binary[i] == 1 and y_pred_binary[i] == 0):
        error1to0 += 1
        
print('error0to1 = ',error0to1, '; label is 0')
print('error1to0 = ',error1to0, '; label is 1')
print('total error = ',error0to1+error1to0,'percentError = ', 100*(error0to1+error1to0)/N)
print('2000 test cases: 1000 label 0; 1000 label 1')

  max and min 
0.9997665
0.092390984
(2000, 8)
(2000,)
(2000, 1)
** CHECK Average BCE Loss for multiple samples: [0.2466]
Accuracy Score: 0.865
error0to1 =  270 ; label is 0
error1to0 =  0 ; label is 1
total error =  270 percentError =  13.5
2000 test cases: 1000 label 0; 1000 label 1


In [169]:
print(DBset_test.shape)
print(DBy.shape)
print(DBy[0:16])
#print(y_pred_binary.shape)
#print(y_pred_binary[0:16])

print('Compare between DBy and y_pred_binary')

errorcase=[0, 0, 0, 0, 0, 0, 0, 0]

for i in range(0,2000):
    index = (i % 8)
    if (index == 0 and DBy[i] != y_pred_binary[i] ): 
        errorcase[index] = errorcase[index]+1
    if (index == 1 and DBy[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
    if (index == 2 and DBy[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
    if (index == 3 and DBy[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
    if (index == 4 and DBy[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
    if (index == 5 and DBy[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
    if (index == 6 and DBy[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
    if (index == 7 and DBy[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
print(errorcase)

(2000, 8)
(2000,)
[1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0.]
Compare between DBy and y_pred_binary
[0, 0, 0, 0, 12, 11, 0, 247]


In [ ]:
# To check based on the training dataset

In [170]:
# To get the results of the ANN using train dataset
y_pred = ANN_model.predict(DBset)

print('  max and min ')
print(np.max(y_pred))
print(np.min(y_pred))

epsilon = 1e-15  # Small value to prevent log(0)
y_pred = np.clip(y_pred, epsilon, 1 - epsilon)  # Clip probabilities

print(DBset.shape)
print(DBy.shape)
print(y_pred.shape)

total_loss = binary_cross_entropy_check(DBy, y_pred)
print(f"** CHECK Average BCE Loss for multiple samples: {total_loss}")

# To check on accuracy
# first, convert the elements in y_pred so that negative becones epsilon, largest is 1 - epsilon
epsilon = 1e-15  # Small value to prevent log(0)
y_pred = np.clip(y_pred, epsilon, 1 - epsilon)  # Clip probabilities
N=len(DBy)
threshold = 0.5
y_pred_binary = (y_pred >= threshold).astype(int)

accuracy = accuracy_score(DBy,y_pred_binary)
print("Accuracy Score:",accuracy)

#Count error cases:
#
test_y_binary = DBy
error0to1 = 0  # test_y is 0
error1to0 = 0  # test_y is 1
for i in range(N):
    if (test_y_binary[i] == 0 and y_pred_binary[i] == 1):
        error0to1 += 1
    if (test_y_binary[i] == 1 and y_pred_binary[i] == 0):
        error1to0 += 1
        
print('error0to1 = ',error0to1, '; label is 0')
print('error1to0 = ',error1to0, '; label is 1')
print('total error = ',error0to1+error1to0,'percentError = ', 100*(error0to1+error1to0)/N)
print('2000 training cases: 1000 label 0; 1000 label 1')

  max and min 
0.9997596
0.09228331
(2000, 8)
(2000,)
(2000, 1)
** CHECK Average BCE Loss for multiple samples: [0.248]
Accuracy Score: 0.863
error0to1 =  274 ; label is 0
error1to0 =  0 ; label is 1
total error =  274 percentError =  13.7
2000 training cases: 1000 label 0; 1000 label 1


In [172]:
print('Training dataset: Compare between DBy and y_pred_binary')

errorcase=[0, 0, 0, 0, 0, 0, 0, 0]

for i in range(0,2000):
    index = (i % 8)
    if (index == 0 and DBy[i] != y_pred_binary[i] ): 
        errorcase[index] = errorcase[index]+1
    if (index == 1 and DBy[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
    if (index == 2 and DBy[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
    if (index == 3 and DBy[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
    if (index == 4 and DBy[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
    if (index == 5 and DBy[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
    if (index == 6 and DBy[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
    if (index == 7 and DBy[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
print(errorcase)
print(np.sum(errorcase))

Training dataset: Compare between DBy and y_pred_binary
[0, 0, 0, 0, 20, 8, 0, 246]
274


In [40]:
# NO USE BELOW

#Define ANOTHER ANN model
input_dim = 16384
ANNmodel2 = keras.Sequential([
    layers.Dense(512, activation='relu', input_shape=(1500,input_dim)),
    layers.Dense(128, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid') # Output layer for binary classification
])

# compile with cross entropy
ANNmodel2.compile(optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'])

In [ ]:
# training

start_time = time.time()

history = ANNmodel2.fit(x_train, y_train, epochs=30, batch_size=32,verbose = 2)

end_time = time.time()
exe_time = end_time - start_time
print("Execution time: ", exe_time)

In [ ]:
scores = ANNmodel2.evaluate(test_x,test_y,verbose = 2)

In [55]:
# test code
import numpy as np
from keras.losses import binary_crossentropy
import keras.backend as K

# Example true labels and predicted probabilities
y_true = np.array([0, 1, 1, 0, 1])
y_pred = np.array([0.1, 0.9, 0.8, 0.2, 0.7])

# Compute Binary Cross-Entropy using NumPy
def binary_cross_entropy(y_true, y_pred):
    bce = -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
    return bce

bce_loss = binary_cross_entropy(y_true, y_pred)
print(f"***Binary Cross-Entropy Loss (function): {bce_loss}")

#===========================================================================
def binary_cross_entropy_np(y_true, y_pred):
    """
    Calculates Binary Cross-Entropy loss for multiple samples using NumPy.
    y_true: NumPy array of actual labels (0s and 1s)
    y_pred: NumPy array of predicted probabilities (between 0 and 1)
    """
    epsilon = 1e-15  # Small value to prevent log(0)
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)  # Clip probabilities
    loss = -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
    return loss

total_loss = binary_cross_entropy_np(y_true, y_pred)
print(f"--Average BCE Loss for multiple samples: {total_loss}")
#--------------------------------------------------------------------------

# Compute Binary Cross-Entropy using Keras
# DOES NOT WORK
# bce_loss_keras = binary_crossentropy(K.constant(y_true), K.constant(y_pred)).numpy()
# print(f"Binary Cross-Entropy Loss (Keras): {bce_loss_keras}")

***Binary Cross-Entropy Loss (function): 0.20273661557656092
--Average BCE Loss for multiple samples: 0.20273661557656092


In [84]:
aaa = np.array([0, 1, 1, -2.5, 1 , 3.4])
bbb = np.array([0.1, 0.9, 0.8, 0.2, 0.7])

print(max(aaa))
print(min(aaa))

3.4
-2.5


In [ ]:
    DBset = np.empty((0,4))
    DBset_tmp = np.array([1,2,3,4])
    DBset = np.append(DBset,[DBset_tmp],axis=0)
    print(DBset)
    
    DBset_tmp = np.array([5,6,7,8])
    DBset = np.append(DBset,[DBset_tmp],axis=0)

    print(DBset)
    print('========================')
    DBset_tmp = np.array([9,10,11,12])
    DBset = np.append(DBset,[DBset_tmp],axis=0)

    print(DBset)